In [1]:
import tensorflow as tf
import keras
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, BatchNormalization
from keras.models import Model

In [2]:
train_data_dir = './Train_data/'
validation_data_dir = './Validation_data/'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    #horizontal_flip=True,
    fill_mode='reflect'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # ResNet152 expects input shape of (224, 224, 3)
    batch_size=16,#32
    class_mode='categorical'
)


Found 8006 images belonging to 10 classes.


In [3]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),  # ResNet152 expects input shape of (224, 224, 3)
    batch_size=16,#32
    class_mode='categorical'
)


Found 2000 images belonging to 10 classes.


In [4]:
import tensorflow as tf
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import Precision, Recall
from keras.layers import GlobalAveragePooling2D
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output) #GlobalAverage instead of flatten()
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)  # Reduced dropout rate
predictions = Dense(10, activation='softmax')(x)

In [5]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with custom metrics
model.compile(optimizer=Adam(learning_rate=0.0001),#0.0001
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6)#1e-7

In [7]:
checkpoint_best = ModelCheckpoint("./best_models/bestfinalmodel.keras", monitor='val_accuracy', save_best_only=True, mode='max')

checkpoint_last = ModelCheckpoint("./last_models/lastfinalmodel.keras", save_best_only=False)

# Train the model with early stopping
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=25,
                    callbacks=[early_stopping,reduce_lr,checkpoint_best,checkpoint_last])

# Save the model
model.save("./models/bestmodelfinetuned25.keras")

Epoch 1/25
501/501 [==============================] - 97s 193ms/step - loss: 0.4802 - accuracy: 0.8511 - precision: 0.8939 - recall: 0.8268 - val_loss: 2.7667 - val_accuracy: 0.1470 - val_precision: 0.1549 - val_recall: 0.0295 - lr: 1.0000e-04
Epoch 2/25
501/501 [==============================] - 93s 185ms/step - loss: 0.1800 - accuracy: 0.9445 - precision: 0.9531 - recall: 0.9364 - val_loss: 0.8785 - val_accuracy: 0.7775 - val_precision: 0.8005 - val_recall: 0.7605 - lr: 1.0000e-04
Epoch 3/25
501/501 [==============================] - 97s 194ms/step - loss: 0.1357 - accuracy: 0.9570 - precision: 0.9644 - recall: 0.9502 - val_loss: 0.6007 - val_accuracy: 0.8810 - val_precision: 0.8901 - val_recall: 0.8750 - lr: 1.0000e-04
Epoch 4/25
501/501 [==============================] - 97s 193ms/step - loss: 0.1339 - accuracy: 0.9564 - precision: 0.9624 - recall: 0.9529 - val_loss: 0.4448 - val_accuracy: 0.8995 - val_precision: 0.9134 - val_recall: 0.8960 - lr: 1.0000e-04
Epoch 5/25
501/501 [====

In [1]:
from keras.models import load_model

# Load the model
loaded_model = load_model('./models/bestmodelfinetuned25.keras')


In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import csv

# Define class labels
class_labels = ['Crane', 'Crow', 'Egret', 'Kingfisher', 'Myna', 'Peacock', 'Pitta', 'Rosefinch', 'Tailorbird', 'Wagtail']

# Directory containing your images
predict_dir = './Eval_data/'

# List all image files in the directory
image_files = os.listdir(predict_dir)

# Open a CSV file to write the results
with open('submissions.csv', mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', 'Target_name','Target_num'])
    # Loop through each image file and make predictions
    for img_file in image_files:
        # Load and preprocess the image
        img_path = os.path.join(predict_dir, img_file)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array /= 255.0  # Normalize pixel values

        # Make prediction
        prediction = loaded_model.predict(img_array)

        # Get the predicted class label
        predicted_class_index = np.argmax(prediction)
        predicted_class = class_labels[predicted_class_index]

        # Print the predicted class label
        print(f"ID: {img_file}, Target_name: {predicted_class},Target_num: {predicted_class_index}")

        # Write the result to the CSV file
        writer.writerow([img_file[:-4], predicted_class,predicted_class_index])

print("Predictions saved to submissions.csv")

1/1 [==============================] - 8s 8s/step
ID: 0.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 38ms/step
ID: 1.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 25ms/step
ID: 10.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 23ms/step
ID: 100.jpg, Target_name: Crow,Target_num: 1
1/1 [==============================] - 0s 29ms/step
ID: 1000.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 28ms/step
ID: 1001.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 25ms/step
ID: 1002.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 21ms/step
ID: 1003.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 20ms/step
ID: 1004.jpg, Target_name: Crane,Target_num: 0
1/1 [==============================] - 0s 25ms/step
ID: 1005.jpg, Target_name: Crane,Target_num: 0
1/1 [=================

In [10]:
test_data_dir = './test_data/'
saved_model = tf.keras.models.load_model("./models/bestmodelfinetuned25.keras")
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),  # Adjust according to your model's input shape
    batch_size=32,
    class_mode='categorical',  # Set to 'binary' for binary classification
    shuffle=False  # Ensure that predictions are in the same order as the ground truth labels
)

# Evaluate the model on the test set
test_results = saved_model.evaluate(test_generator)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

Found 1998 images belonging to 10 classes.
63/63 [==============================] - 11s 152ms/step - loss: 0.1767 - accuracy: 0.9560 - precision: 0.9597 - recall: 0.9545
Test Loss: 0.17667154967784882
Test Accuracy: 0.955955982208252
